In [1]:
import pandas as pd
import numpy as np
import json

In [2]:
with open('./companies.jsonl', 'r', encoding='utf-8') as file:
    companies_data = (json.loads(line[:-1]) for line in file.readlines())
with open('./factories.jsonl', 'r', encoding='utf-8') as file:
    factories_data = (json.loads(line[:-1]) for line in file.readlines())

In [3]:
companies_df = pd.DataFrame(companies_data)
factories_df = pd.DataFrame(factories_data)

In [4]:
companies_df['type'] = companies_df['branches'].apply(lambda _: 'شركة')
factories_df['type'] = factories_df['branches'].apply(lambda _: 'مصنع')
df = pd.concat([companies_df, factories_df], ignore_index=True)

In [6]:
employers = df['employers'].to_list()

In [7]:
def parse_list(items: list) -> dict:
    return {list(item.keys())[0]: list(item.values())[0] for item in items}

In [8]:
employers_df = pd.DataFrame(list(map(parse_list, employers)))

In [9]:
phones_df = df['phones'].apply(pd.Series)

In [10]:
phones_df.replace(np.nan, '', inplace=True)

In [11]:
phones_df.columns = [f'tel{idx}' for idx in range(1, 5)]

In [12]:
phones_df['tels'] = phones_df['tel1'] + "-" + phones_df['tel2'] + \
    "-" + phones_df['tel3'] + "-" + phones_df['tel4']

In [13]:
phones_df = phones_df['tels'].astype(str)

In [14]:
phones_df.replace([r'tel:', r'\-{2,}'], ['', '-'], regex=True, inplace=True)

In [15]:
phones_df = phones_df.str.split(pat='-', expand=True)

In [16]:
phones_df.columns = [f'phone_{idx}' for idx in range(1, 9)]

In [17]:
contact_details_df = df['contact_details'].apply(pd.Series)

In [18]:
emails_df = contact_details_df['البريد الالكتروني'].apply(pd.Series)

In [19]:
emails_df['emails'] = emails_df[0] + ' - ' + emails_df[1]

In [20]:
emails_df = emails_df['emails']

In [21]:
emails_df = emails_df.str.replace(r'mailto:', '').str.split(" - ", expand=True)

In [22]:
emails_df.columns = [f'email_{idx}' for idx in range(1, 5)]

In [23]:
websites_df = contact_details_df['الموقع الالكتروني'].apply(pd.Series)

In [24]:
def address_from_branch(branches):
    city, address = list(" " * 2)
    try:
        key = list(branches['branches'].keys())[0]
        full_address: str = branches['branches'][key]['address']
        arr = full_address.split(':')
        if len(arr) == 2:
            city, address = arr
        else:
            address = full_address
        return city.strip(), address.strip()
    except:
        return city, address

In [26]:
branches = df[['title', 'branches']]

In [27]:
addresses = list(map(address_from_branch, branches.to_dict('records')))
addresses_df = pd.DataFrame(addresses)

In [28]:
addresses_df.columns = ['city', 'address']

In [29]:
df = pd.concat([df, employers_df, phones_df, emails_df, websites_df, addresses_df],
               axis='columns')

In [30]:
df = df[[
    'type',
    'title',
    'category',
    'description',
    'city',
    'address',
    0,
    'email_1',
    'email_2',
    'email_3',
    'email_4',
    'phone_1',
    'phone_2',
    'phone_3',
    'phone_4',
    'phone_5',
    'phone_6',
    'phone_7',
    'phone_8',
    'رئيس مجلس الإدارة',
    'مدراء آخرون',
    'المدير العام',
    'مدير تسويق',
    'مدير التصدير',
    'مدير المبيعات',
    'مدير مشتريات',
    'branches',
    'link',
]]

In [31]:
df.rename({0: 'website'}, inplace=True, axis='columns')

In [32]:
df['city'].replace(['االقاهرة', 'القاهره'], ['القاهرة']
                   * 2, inplace=True, regex=True)

In [33]:
df['city'].replace(['اا', 'ة', 'أ', 'آ', 'إ', 'ـ'],
                   ['ا', 'ه', 'ا', 'ا', 'ا', ''],
                   inplace=True,
                   regex=True)

In [34]:
df['city'] = df['city'].str.strip()

In [43]:
df = df.dropna(thresh=len(df)*0.1, axis='columns')

In [44]:
df

,type,title,category,description,city,address,website,email_1,email_2,phone_1,phone_2,phone_3,رئيس مجلس الإدارة,مدراء آخرون,المدير العام,مدير تسويق,مدير المبيعات,مدير مشتريات,branches,link
0,شركة,السويس لمهمات السلامة المهنية,الصناعات النسيجية واليونيفورم,تصنيع ملابس وأحذية الأمان والسلامة المهنية وال...,السويس,المنطقة الصناعية – العين السخنة,http://www.ssoegypt.com,NaN,NaN,,,None,حاتم المجد,المدير المالي : محيي عطية / المدير التجاري : أ...,NaN,NaN,NaN,NaN,{'المصنع': {'address': 'السويس : المنطقة الصنا...,https://www.egydir.com/ar/1413/%D8%A7%D9%84%D8...
1,شركة,السيسي للسجاد,الصناعات النسيجية واليونيفورم,بيع سجاد,القاهره,15 ا شارع المنصور محمد الزمالك,NaN,NaN,NaN,01223431969,01223431985,,NaN,NaN,مجدي فتحي,NaN,NaN,NaN,{'الادارة': {'address': 'القاهرة: 15 ا شارع ال...,https://www.egydir.com/ar/10163/%D8%A7%D9%84%D...
2,شركة,الزين والزينة,الصناعات النسيجية واليونيفورم,تصنيع جميع أنواع الزي الموحد ( الفنادق – المست...,القاهره,15 ش هدايا حسن متفره من ش أحمد زكي - المعادي,NaN,NaN,NaN,01211621669,01153657965,01098312025,خالد خطاب,NaN,NaN,NaN,NaN,NaN,{'الادارة': {'address': 'القاهرة : 15 ش هدايا ...,https://www.egydir.com/ar/1319/%D8%A7%D9%84%D8...
3,شركة,الشركه المصريه للصباغه – ازغل تكس,الصناعات النسيجية واليونيفورم,انتاج اقمشه التريكو بكافه انواعها – صباغه جميع...,القاهره,7 ش مجمع المصانع – السواح,NaN,NaN,NaN,01144445088,01224792811,,احمد حسين ابراهيم الازغل,NaN,NaN,محمد احمد حسين ازغل,NaN,NaN,{'المصنع': {'address': 'القاهره : 7 ش مجمع الم...,https://www.egydir.com/ar/11861/%D8%A7%D9%84%D...
4,شركة,الشرقاوي لمستلزمات الملابس,الصناعات النسيجية واليونيفورم,مستلزمات صناعه الملابس الجاهزه,القاهره,10 ش رشدي عابدين,NaN,NaN,NaN,01227874196,,None,عادل نجاتي,NaN,NaN,NaN,NaN,NaN,{'الادارة': {'address': 'القاهرة: 10 ش رشدي عا...,https://www.egydir.com/ar/10164/%D8%A7%D9%84%D...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11794,مصنع,EWB,صناعات التشييد والبناء,تصنيع محابس نحاسية,دمياط الجديده,قطعة 3 بلوك م – المطقة الصناعية الأولى,http://www.ewb.com.eg,NaN,NaN,,,None,دسوقي طه عراب,NaN,NaN,NaN,بوسي الحديدى,NaN,{'المصنع': {'address': 'دمياط الجديدة : قطعة 3...,https://www.egydir.com/ar/9399/EWB/about
11795,مصنع,أنظمة المباني المعدنية المتطورة ( سابس ),صناعات التشييد والبناء,تصنيع منشأت معدنية,بدر,المنطقة الصناعية الثانية,http://www.sabs.com.eg,info@sabs.com.eg,marketing@sabs.com,,,None,خالد أبو الفتوح,المدير المالي : علاء أبو العنين,NaN,أدهم أبو الفتوح,NaN,NaN,{'المصنع': {'address': 'بدر : المنطقة الصناعية...,https://www.egydir.com/ar/45/%D8%A3%D9%86%D8%B...
11796,مصنع,ابسد,صناعات التشييد والبناء,أعمال رخام وجرانيت اعمال الومونيوم اعمال متخصص...,الاقصر,شارع سعد زغلول عمارة الملاواني,NaN,NaN,NaN,01229114944,01223459298,,عاطف نور,NaN,NaN,NaN,NaN,NaN,{'الادارة': {'address': 'الأقصر: شارع سعد زغلو...,https://www.egydir.com/ar/54/%D8%A7%D8%A8%D8%B...
11797,مصنع,Egy Gate,صناعات التشييد والبناء,ابواب المداخل الزجاج الفوتوسل – ماكينات ذراع ل...,القاهره,9 ش محمد فريد – مصر الجديدة,http://www.egy-gate.net,info@egy-gate.net,egypt@egy-gate.net,01005583368,01022203320,01068803250,اشرف عبد المقصود,NaN,NaN,NaN,NaN,NaN,{'الادارة': {'address': 'القاهرة : 9 ش محمد فر...,https://www.egydir.com/ar/9034/Egy-Gate/about
